# Burn Area Index

The Burn Area Index (BAI) is a measure in the red to near-infrared spectrum that emphasizes charcoal signal in post-fire imagery. It measures the spectral distance from exach pixel to a reference spectral point. The brighter pixels represent burned areas.

The following formula is used to calculate BAI:

![alt text](bai.png "BAI Formula")

### Setting the Libraries

In [ ]:
import os
import rasterio

### Setting the paths

In [ ]:
inpath = r"/Users/Jenny/Downloads/Imagery/Images/"

### Calculating the BAI

In [ ]:
# Function to calculate the Burn Area Index for each raster
def bai_rast(inpath, outpath):
    
    # Opens the raster for the calculation
    with rasterio.open(inpath) as src:
        brn_data = src.read()
       
    # BAI Formula
    brn_bai = (1 / ((.1-brn_data[2])**2)+((.06-brn_data[3])**2))
    
    # Sets up the metadata for the output raster
    with rasterio.open(inpath) as src:
        brn_data_ras = src.read()
        brn_meta = src.profile

    brn_transform = brn_meta["transform"]
    brn_crs = brn_meta["crs"]
    brn_meta['count'] = 1
    brn_meta['dtype'] = 'float64'

    # Saves out the raster according to the below criteria
    with rasterio.open(outpath, 'w', **brn_meta) as dst:
        dst.write(brn_bai, 1)

# Function to select files to calculate the BAI on
def compare():
    ending_list = []
    for root, dirs, files in os.walk(inpath):
        for filename in files:
            if filename.endswith("reproj.tif"):
                print('File name is: {0}'.format(filename))
                ending_list.append(filename.split('_',1)[0])

    unique_ending = list(set(ending_list))
    unique2 = unique_ending
    
    # Selects the files using a unique identifier and sets up the out paths
    for unique in unique_ending:
        for root, dirs, files in os.walk(inpath):
            for filename in files:
                if filename.startswith(unique):
                    files_unique = [x for x in files if x.startswith(unique)]
                    file_paths = [x for x in files_unique if x.endswith("reproj.tif")]
                    outpath = inpath + unique + "_bai.tif"
                    if unique != False:
                        os.chdir(inpath)
                        bai_rast(file_paths[0], outpath)
                else:
                    print("N/A")

compare()